In [5]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from sys import path
import matplotlib.pyplot as plt
import pymmwr as pm
from datetime import datetime
from datetime import date
from datetime import timedelta

In [6]:
'''Import locations'''

locations = pd.read_csv('./locations.csv',skiprows=0) 
locations = locations.drop([0]) #skip first row (national ID)
print("Number of Locations:", len(locations))
locations.head()

Number of Locations: 52


,abbreviation,location,location_name,population,Unnamed: 4,count_rate1,count_rate2,count_rate2p5,count_rate3,count_rate4,count_rate5
1,AL,01,Alabama,5063778,NaN,51,101,127,152,203,253
2,AK,02,Alaska,711426,NaN,7,14,18,21,28,36
3,AZ,04,Arizona,7341018,NaN,73,147,184,220,294,367
4,AR,05,Arkansas,3041878,NaN,30,61,76,91,122,152
5,CA,06,California,38886551,NaN,389,778,972,1167,1555,1944


In [7]:
'''Extract hospitalization data'''

actual_data = pd.read_csv('./COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW_.csv') 
actual_data = actual_data[['date','state','previous_day_admission_influenza_confirmed']].sort_values(['state','date'])
actual_data['date'] = pd.to_datetime(actual_data['date'])
actual_data.head()

,date,state,previous_day_admission_influenza_confirmed
33828,2020-03-23,AK,NaN
28607,2020-03-24,AK,NaN
34678,2020-03-25,AK,NaN
32325,2020-03-26,AK,NaN
26758,2020-03-27,AK,NaN


In [8]:
'''Map location numbers to state abbreviations'''

location_to_state = {}
for index, row in locations.iterrows():
    location_number = row['location']
    abbreviation = row['abbreviation']
    location_to_state.update({location_number: abbreviation})
print(location_to_state)

{'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}


In [9]:
'''Testing. Get a single state's actual data.'''

NY_actual_data = pd.DataFrame(actual_data[actual_data['state'] == 'NY'])
NY_actual_data
daily_count = NY_actual_data.loc[NY_actual_data['date'] == '2023-05-13'].values[0, 2]
daily_count

7.0

In [10]:
'''Convert the forecast data files into a dataframe'''

from os import listdir
from os.path import isfile, join

forecast_path = './LosAlamos_NAU-CModel_Flu/'
forecast_files = [file for file in listdir(forecast_path) if isfile(join(forecast_path, file))]
forecast_files.sort()

dataframes = []
for file in forecast_files:
    df = pd.read_csv(forecast_path + file)
    df['target_end_date'] = pd.to_datetime(df['target_end_date'])
    dataframes.append(df)

all_forecasts = pd.concat(dataframes)
all_forecasts

,reference_date,target,horizon,target_end_date,location,output_type,output_type_id,value
0,2023-10-21,wk inc flu hosp,-1,2023-10-14,11,quantile,0.01,0.0
1,2023-10-21,wk inc flu hosp,-1,2023-10-14,11,quantile,0.025,0.0
2,2023-10-21,wk inc flu hosp,-1,2023-10-14,11,quantile,0.05,0.0
3,2023-10-21,wk inc flu hosp,-1,2023-10-14,11,quantile,0.1,0.0
4,2023-10-21,wk inc flu hosp,-1,2023-10-14,11,quantile,0.15,0.0
...,...,...,...,...,...,...,...,...
5931,2024-05-04,wk flu hosp rate change,3,2024-05-25,US,pmf,stable,1.0
5932,2024-05-04,wk flu hosp rate change,3,2024-05-25,US,pmf,increase,0.0
5933,2024-05-04,wk flu hosp rate change,3,2024-05-25,US,pmf,large_increase,0.0
5934,2024-05-04,wk flu hosp rate change,3,2024-05-25,US,pmf,decrease,0.0


In [110]:
'''Testing. Get a single state's forecast data.'''

MI_forecast_data = all_forecasts[all_forecasts['location'] == '26']
MI_forecast_data = MI_forecast_data[MI_forecast_data['output_type'] == 'quantile']
MI_forecast_data['output_type_id'] = MI_forecast_data['output_type_id'].astype(float)
MI_forecast_data.to_csv('./mi-data-test.csv')

print(all(isinstance(x, float) for x in MI_forecast_data['value']))

print(all(isinstance(x, float) for x in MI_forecast_data['output_type_id']))


True
True


In [12]:
'''Define the Interval Score and Weighted Interval Score functions'''
'''https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7880475/'''

def IS(alpha, predL, predU):
    
    return lambda y: (predU - predL) + 2/alpha*(y < predL)*(predL - y) + 2/alpha*(y > predU)*(y-predU)

def WIS(y_obs, qtlMark, predQTL):
    
    is_well_defined = np.mod(len(qtlMark), 2) != 0
    
    NcentralizedQT = (len(qtlMark)-1)//2 + 1
    
    alpha_list = np.zeros(NcentralizedQT)
    weight_list = np.zeros(NcentralizedQT)
    
    for i in range(NcentralizedQT):  
        is_well_defined = is_well_defined & (np.abs(-1.0 + qtlMark[i] + qtlMark[-1-i]) < 1e-8)
        alpha_list[i] = 1 - (qtlMark[-1-i] - qtlMark[i])
        weight_list[i] = alpha_list[i]/2
        
    if is_well_defined:
        #print(alpha_list)
        #print(qtlMark)
        #print(NcentralizedQT)
        
        output = 1.0/2 * np.abs(y_obs - predQTL[NcentralizedQT - 1])
        
        for i in range(NcentralizedQT - 1):
            output += weight_list[i] * IS(alpha_list[i], predQTL[i], predQTL[-1 - i])(y_obs)
            #print(alpha_list[i], predQTL[i],predQTL[-1-i])
            
        return output/(NcentralizedQT - 1/2)
    
    else:
        print('Check the quantile marks: either no median defined, or not in symmetric central QTL form.')

In [15]:
'''Testing to see if all states/locations are represented in our forecast data files. '''
test_data = pd.read_csv('./LosAlamos_NAU-CModel_Flu/2023-10-21-LosAlamos_NAU-CModel_Flu.csv')
unique_states = len(test_data['location'].unique())
unique_states

53

In [117]:
def get_target_dates_list(forecast_df):
    return forecast_df['target_end_date'].unique()


def get_state_actual_data(state_code):
    state_abbrev = location_to_state[str(state_code).zfill(2)]
    return actual_data[actual_data['state'] == state_abbrev]


def one_state_one_week_WIS_scores(forecast_df, state_code):
    state_code = str(state_code).zfill(2)
    quantiles = np.zeros((23,4))
    wis = np.zeros(4)

    state_actual_data = get_state_actual_data(state_code)

    target_dates = get_target_dates_list(forecast_df)
    print('Target Dates:', target_dates)

    for n_week_ahead in range(4):
        target_date = target_dates[n_week_ahead]


        # Sum the daily reported cases to find the weekly observation.
        observation = 0
        for i in range(7):
            target_date = str(date.fromisoformat(target_date) + timedelta(days=-i))
            daily_count = state_actual_data.loc[state_actual_data['date'] == target_date].values[0, 2]
            observation += daily_count

        print('Observation:', observation)

        df_state = forecast_df[forecast_df['location'].to_numpy() == state_code]
        df_state_forecast = df_state[df_state['target_end_date'] == target_dates[n_week_ahead]]
        quantiles = df_state_forecast['output_type_id'].astype(float).to_numpy()
        predictions = df_state_forecast['value'].astype(float).to_numpy()
        
        try:
            wis[n_week_ahead] = np.round(
                                np.sum(np.nan_to_num(
                                    WIS(observation, 
                                        quantiles, 
                                        predictions))),
                                        2)
        except:
            # if an error occurs, print the corresponding data
            print("An error occured. Output will include a row of 0's.")
            print(df_state_forecast)
            print(quantiles)
            print(predictions)

    # Send scores to csv
    one_week_scores = {'state_code': state_code, 'state_abbrev': location_to_state[state_code], 'date': target_dates[0], '1wk_WIS': wis[0],'2wk_WIS': wis[1], '3wk_WIS': wis[2], '4wk_WIS': wis[3]}

    return one_week_scores


def one_state_all_scores(state_code):
    state_code = str(state_code).zfill(2)
    
    state_df = pd.DataFrame(columns=[])

    forecast_path = './LosAlamos_NAU-CModel_Flu/'
    forecast_files = [file for file in listdir(forecast_path) if isfile(join(forecast_path, file))]
    forecast_files.sort()

    for file in forecast_files:
        all_forecast_data = pd.read_csv(forecast_path + file)
        all_forecast_data = all_forecast_data[all_forecast_data['output_type'] == 'quantile']

        weekly_scores = one_state_one_week_WIS_scores(all_forecast_data, state_code)
        state_df = pd.concat([state_df, pd.DataFrame([weekly_scores])], ignore_index=True)
    
    # Export to CSV
    state_csv_path = './mcmc_accuracy_results/' + location_to_state[state_code] + '.csv'
    state_df.to_csv(state_csv_path)


In [118]:
''' Testing '''

forecast_path = './LosAlamos_NAU-CModel_Flu/2023-10-21-LosAlamos_NAU-CModel_Flu.csv'
all_forecast_data = pd.read_csv(forecast_path)
all_forecast_data = all_forecast_data[all_forecast_data['output_type'] == 'quantile']
AZ_forecast_data = all_forecast_data[all_forecast_data['location'] == '04']

# Check if all values are floats. 
all(isinstance(x, float) for x in AZ_forecast_data['value'])

True

In [119]:
''' Testing the one state function '''

one_state_one_week_WIS_scores(all_forecast_data, 4)

Target Dates: ['2023-10-14' '2023-10-21' '2023-10-28' '2023-11-04' '2023-11-11']
Observation: 24.0
Observation: 34.0
Observation: 35.0
Observation: 46.0

WIS: [13.96 20.47 26.59 32.46]


{'state_code': '04',
 'state_abbrev': 'AZ',
 'date': '2023-10-14',
 '1wk_WIS': 13.96,
 '2wk_WIS': 20.47,
 '3wk_WIS': 26.59,
 '4wk_WIS': 32.46}

In [120]:
one_state_all_scores('04')

Target Dates: ['2023-10-14' '2023-10-21' '2023-10-28' '2023-11-04' '2023-11-11']
Observation: 24.0
Observation: 34.0
Observation: 35.0
Observation: 46.0

WIS: [13.96 20.47 26.59 32.46]
Target Dates: ['2023-10-21' '2023-10-28' '2023-11-04' '2023-11-11' '2023-11-18']
Observation: 34.0
Observation: 35.0
Observation: 46.0
Observation: 85.0

WIS: [11.13  7.98 17.58 64.31]
Target Dates: ['2023-10-28' '2023-11-04' '2023-11-11' '2023-11-18' '2023-11-25']
Observation: 35.0
Observation: 46.0
Observation: 85.0
Observation: 145.0

WIS: [ 12.13  18.74  56.58 124.31]
Target Dates: ['2023-11-11' '2023-11-18' '2023-11-25' '2023-12-02']
Observation: 85.0
Observation: 145.0
Observation: 176.0
Observation: 232.0

WIS: [ 28.2   79.27  98.32 141.61]
Target Dates: ['2023-11-18' '2023-11-25' '2023-12-02' '2023-12-09']
Observation: 145.0
Observation: 176.0
Observation: 232.0
Observation: 323.0

WIS: [ 88.2  110.27 154.32 232.61]
Target Dates: ['2023-11-25' '2023-12-02' '2023-12-09' '2023-12-16']
Observation: 

In [121]:
for state in location_to_state.keys():
    one_state_all_scores(state)

Target Dates: ['2023-10-14' '2023-10-21' '2023-10-28' '2023-11-04' '2023-11-11']
Observation: 22.0
Observation: 29.0
Observation: 33.0
Observation: 32.0

WIS: [ 1.36  8.96 12.76 12.07]
Target Dates: ['2023-10-21' '2023-10-28' '2023-11-04' '2023-11-11' '2023-11-18']
Observation: 29.0
Observation: 33.0
Observation: 32.0
Observation: 44.0

WIS: [ 7.97  7.05  5.91 11.62]
Target Dates: ['2023-10-28' '2023-11-04' '2023-11-11' '2023-11-18' '2023-11-25']
Observation: 33.0
Observation: 32.0
Observation: 44.0
Observation: 68.0

WIS: [11.62  6.39 15.13 33.54]
Target Dates: ['2023-11-11' '2023-11-18' '2023-11-25' '2023-12-02']
Observation: 44.0
Observation: 68.0
Observation: 61.0
Observation: 72.0

WIS: [ 9.73 27.92 17.22 27.82]
Target Dates: ['2023-11-18' '2023-11-25' '2023-12-02' '2023-12-09']
Observation: 68.0
Observation: 61.0
Observation: 72.0
Observation: 79.0

WIS: [29.66 21.84 26.24 34.09]
Target Dates: ['2023-11-25' '2023-12-02' '2023-12-09' '2023-12-16']
Observation: 61.0
Observation: 72